# XOR Multiple Inputs/Targets

Testing...

In [1]:
from conx import Network, Layer, SGD

Using Theano backend.


In [2]:
net = Network("XOR", 2, 2, 1, activation="sigmoid")
net.connect()
dataset = [
    ([0, 0], [0]),
    ([0, 1], [1]),
    ([1, 0], [1]),
    ([1, 1], [0])
]

In [3]:
net.set_dataset(dataset)

Split dataset into:
   4 train inputs
   0 test inputs
Input Summary:
   length  : 4
   training: 4
   testing : 0
   shape  : (2,)
   range  : (0.0, 1.0)
Target Summary:
   length  : 4
   training: 4
   testing : 0
   shape  : (1,)
   range  : (0.0, 1.0)


In [4]:
net.get_target(0)

[0.0]

In [5]:
net.compile(loss='mean_squared_error',
            optimizer=SGD(lr=0.3, momentum=0.9))

In [6]:
net.config["image_maxdim"] = 50

In [7]:
net.build_widget()

<IPython.core.display.Javascript object>

In [8]:
net.train(epochs=2000, accuracy=1.0, report_rate=25)

Training...
Epoch #   25 | train error 0.24994 | train accuracy 0.50000 | validate% 0.00000
Epoch #   50 | train error 0.24893 | train accuracy 0.50000 | validate% 0.00000
Epoch #   75 | train error 0.24641 | train accuracy 0.50000 | validate% 0.00000
Epoch #  100 | train error 0.23431 | train accuracy 0.50000 | validate% 0.00000
Epoch #  125 | train error 0.19728 | train accuracy 0.50000 | validate% 0.00000
Epoch #  150 | train error 0.15810 | train accuracy 0.50000 | validate% 0.00000
Epoch #  175 | train error 0.14173 | train accuracy 0.75000 | validate% 0.00000
Epoch #  200 | train error 0.13551 | train accuracy 0.50000 | validate% 0.25000
Epoch #  225 | train error 0.13257 | train accuracy 0.50000 | validate% 0.50000
Epoch #  250 | train error 0.13090 | train accuracy 0.50000 | validate% 0.50000
Epoch #  275 | train error 0.12982 | train accuracy 0.50000 | validate% 0.50000
Epoch #  300 | train error 0.12907 | train accuracy 0.50000 | validate% 0.50000
Epoch #  325 | train error 0

In [9]:
net.test()

Testing on training dataset...
# | inputs | targets | outputs | result
---------------------------------------
0 | [0.0, 0.0] | [0.0] | [0.0] | correct
1 | [0.0, 1.0] | [1.0] | [0.5] | X
2 | [1.0, 0.0] | [1.0] | [1.0] | correct
3 | [1.0, 1.0] | [0.0] | [0.5] | X
Total count: 4
Total percentage correct: 0.5


In [10]:
net.propagate_to("input", [0, 1])

[0.0, 1.0]

In [11]:
net.propagate([0.5, 0.5])

[0.50589627]

In [12]:
net.propagate_to("hidden", [1, 0])

[0.7317737340927124, 0.02126365527510643]

In [13]:
net.propagate_to("output", [0.25, 0.75])

[0.49926936626434326]

In [14]:
net.propagate_to("input", [0.25, 0.25])

[0.25, 0.25]

In [15]:
net.propagate_from("input", [1.0, 1.0])

[0.74868131]

In [16]:
net.propagate_from("hidden", [1.0, 0.0])

[0.51548773]

In [17]:
net.test()

Testing on training dataset...
# | inputs | targets | outputs | result
---------------------------------------
0 | [0.0, 0.0] | [0.0] | [0.0] | correct
1 | [0.0, 1.0] | [1.0] | [0.5] | X
2 | [1.0, 0.0] | [1.0] | [1.0] | correct
3 | [1.0, 1.0] | [0.0] | [0.5] | X
Total count: 4
Total percentage correct: 0.5


In [18]:
from conx import Network, Layer, SGD

In [19]:
net = Network("XOR2")
net.add(Layer("input1", shape=1))
net.add(Layer("input2", shape=1))
net.add(Layer("hidden1", shape=10, activation="sigmoid"))
net.add(Layer("hidden2", shape=10, activation="sigmoid"))
net.add(Layer("shared-hidden", shape=5, activation="sigmoid"))
net.add(Layer("output1", shape=1, activation="sigmoid"))
net.add(Layer("output2", shape=1, activation="sigmoid"))

In [20]:
net

<Network name='XOR2'>

In [21]:
net.connect("input1", "hidden1")
net.connect("input2", "hidden2")
net.connect("hidden1", "shared-hidden")
net.connect("hidden2", "shared-hidden")
net.connect("shared-hidden", "output1")
net.connect("shared-hidden", "output2")

In [22]:
net.set_input_layer_order("input1", "input2")
net.set_output_layer_order("output1", "output2")

In [23]:
net.compile(loss='mean_squared_error',
            optimizer=SGD(lr=0.3, momentum=0.9))

In [24]:
net.config["image_maxdim"] = 50
net.build_widget()

<IPython.core.display.Javascript object>

In [25]:
dataset = [
    ([[0],[0]], [[0],[0]]),
    ([[0],[1]], [[1],[1]]),
    ([[1],[0]], [[1],[1]]),
    ([[1],[1]], [[0],[0]])
]

In [26]:
net.set_dataset(dataset)

Split dataset into:
   4 train inputs
   0 test inputs
Input Summary:
   length  : 4
   training: 4
   testing : 0
   shape  : [(1,), (1,)]
   range  : (0.0, 1.0)
Target Summary:
   length  : 4
   training: 4
   testing : 0
   shape  : [(1,), (1,)]
   range  : (0.0, 1.0)


In [27]:
net.config["image_maxdim"] = 50

In [28]:
net.build_widget()

<IPython.core.display.Javascript object>

In [29]:
net["shared-hidden"].colormap = "spring"
net.propagate_from("hidden1", [0.0, 1.0] * 5)

[[0.43286353], [0.37866807]]

In [30]:
net["shared-hidden"].colormap = "hot"
net.propagate_from("hidden1", [1.0, 1.0] * 5, ["shared-hidden"])

[0.13466831, 0.94466233, 0.26798728, 0.54851562, 0.048278283]

In [31]:
net.propagate_to("output1", [[1], [0]])

[0.45758262276649475]

In [32]:
for i in range(20):
    (epoch_count, loss, acc, val_acc) = net.train(epochs=100, verbose=0)
    for index in range(4):
        net.propagate(dataset[index][0])

In [33]:
net.train(epochs=2000, accuracy=1.0, report_rate=25)

Training...
Epoch # 2001 | train error 0.00068 | train accuracy 2.00000 | validate% 1.00000


In [34]:
net.propagate_from("input1", [0.0])

[[0.5], [0.5]]

In [35]:
net.propagate_from("shared-hidden", [0.0] * 5)

[[0.5], [0.5]]

In [36]:
net.propagate_to("hidden1", [[1], [1]])

[0.26811984181404114,
 0.16000720858573914,
 0.7104865908622742,
 0.8802912831306458,
 0.7386482954025269,
 0.8133577108383179,
 0.2863965332508087,
 0.14373797178268433,
 0.17428186535835266,
 0.14051681756973267]

In [37]:
net.test()

Testing on training dataset...
# | inputs | targets | outputs | result
---------------------------------------
0 | [[0.0], [0.0]] | [ 0.  0.] | [ 0.02103656  0.02174782] | correct
1 | [[0.0], [1.0]] | [ 1.  1.] | [ 0.9785282  0.9783079] | correct
2 | [[1.0], [0.0]] | [ 1.  1.] | [ 0.98450744  0.98422539] | correct
3 | [[1.0], [1.0]] | [ 0.  0.] | [ 0.01403455  0.01351558] | correct
Total count: 4
Total percentage correct: 1.0


In [38]:
net.slice_dataset(2)

Slicing dataset 0:2...
Input Summary:
   length  : 2
   training: 2
   testing : 0
   shape  : [(1,), (1,)]
   range  : (0.0, 1.0)
Target Summary:
   length  : 2
   training: 2
   testing : 0
   shape  : [(1,), (1,)]
   range  : (0.0, 1.0)


In [39]:
net.train(epochs=2000, accuracy=1.0, report_rate=25)

Training...
Epoch # 2002 | train error 0.00092 | train accuracy 2.00000 | validate% 1.00000


# Conx model is a Keras Model

In [41]:
from keras.utils.vis_utils import model_to_dot
from IPython.display import HTML

In [42]:
dot = model_to_dot(net.model, rankdir="BT")

In [43]:
HTML(dot.create_svg().decode())